In [2]:
print(set(list("Hello")))

{'o', 'e', 'l', 'H'}


In [ ]:
import heapq
import time

output_state = (('T1', 'T2', 'T3'), ('T4', 'T5', 'T6'), ('T7', 'T8', 'B'))  # B represents the empty tile
output_coordinates = {'T1': (0, 0), 'T2': (0, 1), 'T3': (0, 2),
                      'T4': (1, 0), 'T5': (1, 1), 'T6': (1, 2),
                      'T7': (2, 0), 'T8': (2, 1), 'B': (2,2) }

def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 'B':
                return i,j

# Heuristics h1(n) = 0.
def h1(state):
  return 0

# Heuristics h2(n) = number of tiles displaced from their destined position
def h2(state):
    displaced = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != output_state[i][j]:
                displaced += 1
    return displaced

# Heuristics h3(n) = sum of the Manhattan distance of each tile from the goal position.
def h3(state):
    total_manhattan_distance = 0
    for i in range(3):
        for j in range(3):
            value = state[i][j]
            if value != 'B':
            # if value != 0:
                goal_i, goal_j = output_coordinates[value]
                total_manhattan_distance += abs(i - goal_i) + abs(j - goal_j)
    return total_manhattan_distance

# h4(n) = A heuristics such that h(n) > h∗ (n). we observed that maximum h*(n) in 8-puzzle could 4 so we tried taking h(n) as 6 (h(n)>h*(n)) and verified that overestimating it do not provide optimal solution.
def h4(state):
    total_manhattan_distance = 6
    # for i in range(3):
    #     for j in range(3):
    #         value = state[i][j]
    #         if value != 0:
    #             goal_i, goal_j = divmod(value - 1, 3)
    #             total_manhattan_distance += abs(i - goal_i) + abs(j - goal_j)
    return total_manhattan_distance

# Define the A* search algorithm
def a_star_search(start_state, heuristic):
    explored = set()
    priority_queue = [(heuristic(start_state), start_state)]
    g = {start_state: 0}
    # optimal_path = []
    parent_map = {}
    while priority_queue:
        _, current_state = heapq.heappop(priority_queue)
        explored.add(current_state)

        if current_state == output_state:
            optimal_path = []
            while current_state != start_state:
                optimal_path.append(current_state)
                current_state = parent_map[tuple(map(tuple, current_state))]
            optimal_path.append(start_state)
            optimal_path.reverse()
            # for r in optimal_path:
            #    print(r)
            return explored, current_state, max(zip(g.values(), g.keys()))[0] ,optimal_path  #

        for neighbor_state in get_neighbors(current_state):
            if neighbor_state not in explored:
                new_cost = g[current_state] + 1
                if neighbor_state not in g or new_cost < g[neighbor_state]:
                    g[neighbor_state] = new_cost
                    parent_map[tuple(map(tuple, neighbor_state))] = current_state
                    # optimal_path.append(neighbor_state)/
                    heapq.heappush(priority_queue, (new_cost + heuristic(neighbor_state), neighbor_state))
        # print(g)
    # print("g")
    return explored, None , None, None

# Define a function to get neighbor states
def get_neighbors(state):
    neighbors = []
    empty_i, empty_j = find_blank(state)
    for di, dj in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        new_i, new_j = empty_i + di, empty_j + dj
        if 0 <= new_i < 3 and 0 <= new_j < 3:
            new_state = [list(row) for row in state]
            new_state[empty_i][empty_j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[empty_i][empty_j]
            neighbors.append(tuple(tuple(row) for row in new_state))
    return neighbors

# Define a function to print the puzzle state
def print_puzzle(state):
    # for row in state:
    #     print(" ".join(str(tile) for tile in row))
    print(state)
    return

# Define the main function
def main():
    start_state = (('T1', 'T2', 'T3'), ('T4', 'T5', 'B'), ('T7', 'T8', 'T6'))     # sample start state
    # start_state = (('T1', 'T2', 'T3'), ('T4', 'T5', 'T6'), ('B', 'T7', 'T8'))     # very simple start_state
    # start_state = (('T6', 'T7', 'T3'), ('T8', 'T4', 'T2'), ('T1', 'B', 'T5'))       # given in assignment

    heuristics = [h1,h2,h3,h4]
    for heuristic in heuristics:
        print("Heuristic:", heuristic.__name__)
        start_time = time.time()
        explored_states, goal_state_reached, optimal_cost, optimal_path = a_star_search(start_state, heuristic)
        end_time = time.time()

        if goal_state_reached:
            print("Success!")
            print("Start State: ",start_state, sep=" ")
            # print_puzzle(start_state)
            print("Goal State: ",output_state,sep=" ")
            # print_puzzle(goal_state)
            print("Total States Explored: ", len(explored_states))
            print("Total number of states to the optimal path: ", len(explored_states) - 1)
            print("Optimal Path:")
            for state in optimal_path:
              print_puzzle(state)
            print("Optimal Path Cost:", optimal_cost)
            print("Time Taken:", end_time - start_time, "seconds")
        else:
            print("Failure: No solution found.")
            print("Start State: ",start_state)
            print("Goal State: ",output_state,sep=" ")
            # print_puzzle(start_state)/
            print("Total States Explored:", len(explored_states))
        print()

if __name__ == "__main__":
    main()


Heuristic: h1
Success!
Start State:  (('T1', 'T2', 'T3'), ('T4', 'T5', 'B'), ('T7', 'T8', 'T6'))
Goal State:  (('T1', 'T2', 'T3'), ('T4', 'T5', 'T6'), ('T7', 'T8', 'B'))
Total States Explored:  4
Total number of states to the optimal path:  3
Optimal Path:
(('T1', 'T2', 'T3'), ('T4', 'T5', 'B'), ('T7', 'T8', 'T6'))
(('T1', 'T2', 'T3'), ('T4', 'T5', 'T6'), ('T7', 'T8', 'B'))
Optimal Path Cost: 2
Time Taken: 6.29425048828125e-05 seconds

Heuristic: h2
Success!
Start State:  (('T1', 'T2', 'T3'), ('T4', 'T5', 'B'), ('T7', 'T8', 'T6'))
Goal State:  (('T1', 'T2', 'T3'), ('T4', 'T5', 'T6'), ('T7', 'T8', 'B'))
Total States Explored:  2
Total number of states to the optimal path:  1
Optimal Path:
(('T1', 'T2', 'T3'), ('T4', 'T5', 'B'), ('T7', 'T8', 'T6'))
(('T1', 'T2', 'T3'), ('T4', 'T5', 'T6'), ('T7', 'T8', 'B'))
Optimal Path Cost: 1
Time Taken: 5.53131103515625e-05 seconds

Heuristic: h3
Success!
Start State:  (('T1', 'T2', 'T3'), ('T4', 'T5', 'B'), ('T7', 'T8', 'T6'))
Goal State:  (('T1', 'T